In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf

import numpy as np
from npde_master.utils import gen_data, plot_model, eval_model, em_int
from npde_master.npde_helper import build_model, fit_model, save_model, load_model


import os
from os.path import join as oj
import pandas as pd
import numpy as np
from scipy.stats import sem
from collections import defaultdict

from copy import deepcopy

from utils.utils import cwd, set_up_plotting


import time
import datetime
plt = set_up_plotting()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






In [2]:
def create_and_fit_model(t, Y, de_type):
    sess = tf.InteractiveSession()
    de_type = de_type.lower()
    if de_type == 'ode':

    # for ode
        npde = build_model(sess, t, Y, model='ode', sf0=1.0, ell0=np.ones(2), W=6, ktype="id")

        prior_sn = npde.sn.eval()
        # print(f'Prior noise variances are {prior_sn}.')

        npde = fit_model(sess, npde, t, Y, num_iter=500, print_every=50, eta=0.02, plot_=False)

        posterior_sn = npde.sn.eval()

    elif de_type == 'vdp':
    # for vdp
        npde = build_model(sess, t, Y, model='sde', sf0=1.0, ell0=np.ones(2), W=6, ellg0=[1e5], ktype="id")

        prior_sn = npde.sn.eval()

        npde = fit_model(sess, npde, t, Y, Nw=100, num_iter=500, print_every=50, eta=0.02, plot_=False)

        posterior_sn = npde.sn.eval()

    elif de_type == 'sde':
    # for sde 
        npde = build_model(sess, t, Y, model='sde', sf0=1.0, ell0=np.ones(2), W=6, ellg0=[1.0], ktype="id", fix_Z=True)

        prior_sn = npde.sn.eval()

        npde = fit_model(sess, npde, t, Y, Nw=50, num_iter=500, print_every=50, eta=0.01, plot_=False)

        posterior_sn = npde.sn.eval()
    
    else:
        raise NotImplementedError(f"DE type: {de_type} not implemented ")

    del sess
    return npde, prior_sn, posterior_sn

def get_times(overall_t, overall_Y, baseline_obs):
    baseline_obs = np.unique(baseline_obs, axis=0)

    baseline_times = []
    baseline_obs_time_synced = []
    for t_i, y_i in zip(overall_t, overall_Y):
        for ob_i in baseline_obs:
            if (y_i == ob_i).all():
                baseline_times.append(t_i)
                baseline_obs_time_synced.append(list(y_i))
                
    baseline_obs_time_synced = np.asarray(baseline_obs_time_synced)
    baseline_times = np.asarray(baseline_times)
    assert len(baseline_times) == len(baseline_obs_time_synced)
    return baseline_times, baseline_obs_time_synced 


def get_rmse(overall_t, overall_Y, baseline_obs, target, de_type):     
    baseline_times, baseline_obs_synced = get_times(overall_t, overall_Y, baseline_obs)
    npde, prior_sn, posterior_sn = create_and_fit_model([baseline_times], [baseline_obs_synced], de_type)
    
    target_times, target_synced = get_times(overall_t, overall_Y, target)
    
    return eval_model(npde, [target_times], [target_synced], Nw=1)

def get_regression_df(mse_results):
    data_df = defaultdict(list)
    for collab_type, mse_list in mse_results.items():
        baseline = collab_type.replace('-avg-mses', '').replace('-mses', '').replace('_obs','')
        if baseline not in data_df['Baselines']:
            data_df['Baselines'].append(baseline)
        if '-avg-mses' in collab_type:
            avg = np.mean(mse_list)
            se = sem(mse_list)
            data_df['Avg MSE'].append(avg)
            data_df['Stderr'].append(se)
        else:
            stds = np.std(mse_list, axis=1)

            mean_std_mse = np.mean(stds)
            se_std_mse = sem(stds)
            data_df['Std MSE'].append(mean_std_mse)
            data_df['Stderr Std'].append(se_std_mse)
    
    return pd.DataFrame(data=data_df)


def get_mses(collab_type, n, overall_t, overall_Y, baseline_obs, Ts, de_type):
    if 'indiv' in collab_type:
        mses = [get_rmse(overall_t, overall_Y, baseline_obs[i], Ts[i], de_type) for i in range(n) ]

    else:
        mses = []

        baseline_obs = np.unique(baseline_obs, axis=0)
        baseline_times, baseline_obs_synced = get_times(overall_t, overall_Y, baseline_obs)
        npde, prior_sn, posterior_sn = create_and_fit_model([baseline_times], [baseline_obs_synced], de_type)
        
        for i, target in enumerate(Ts):
            target = np.unique(target, axis=0)
            target_times, target_synced = get_times(overall_t, overall_Y, target)
            mses.append(eval_model(npde, [target_times], [target_synced], Nw=1))                                   
    return mses


In [ ]:
def run_regression_analysis(de_type='ODE'):
    results_dir = oj('results', 'DE')
    
    with cwd(results_dir):
        result_datas = []
        for file in os.listdir():
            if file.endswith('.npz') and de_type.upper() in file:
                result_datas.append(np.load(file, allow_pickle=True))

        mse_results = defaultdict(list)
        
        for trial_i in range(n_trials):
            print(f'**** trial index {trial_i+1} **** ')
            print()
            obs = result_datas[trial_i]['obs'].item()
            times = result_datas[trial_i]['times']

            overall_t = result_datas[trial_i]['t'][trajectory_index]
            overall_Y = result_datas[trial_i]['Y'][trajectory_index]
            
            Ts = result_datas[trial_i]['Ts']
            for collab_type, baseline_obs in obs.items():   
#                 if 'rand' in collab_type:
                if 'entropy' in collab_type or 'indiv' in collab_type: 
                    try:
                        mses = get_mses(collab_type, n, overall_t, overall_Y, baseline_obs, Ts, de_type)
                        mse = np.mean(mses)

                        ode_mse_results[collab_type+'-avg-mses'].append(mse)        
                        ode_mse_results[collab_type+'-mses'].append(mses)

                        print(f'****** {collab_type} MSEs: {mses} Avg MSE: {mse} ******')
                    except Exception as inst:
                        print(type(inst))    # the exception instance
                        print(inst.args)     # arguments stored in .args
                        print(inst)          # __str__ allows args to be printed directly,
                                             # but may be overridden in exception subclasses

        regression_dir = 'regression_results'
#         os.makedirs(regression_dir)
        
#         time_str = datetime.datetime.fromtimestamp(time.time()).strftime('%m-%d-%H:%M:%S')

#         regression_df = get_regression_df(mse_results)
#         regression_df.to_latex(oj(regression_dir, f'{de_type}-regression-{time_str}.tex'), index=False) 
#         regression_df.to_csv(oj(regression_dir, f'{de_type}-regression-{time_str}.csv'), index=False) 
    return

In [19]:
n_trials = 5
n = 3
de_type = 'SDE'
trajectory_index = 2

results_dir = oj('results', 'DE')

with cwd(results_dir):
    result_datas = []
    for file in os.listdir():
        if file.endswith('.npz') and de_type.upper() in file:
            result_datas.append(np.load(file, allow_pickle=True))

    mse_results = defaultdict(list)

    for trial_i in range(n_trials):
        print(f'**** trial index {trial_i+1} **** ')
        print()
        obs = result_datas[trial_i]['obs'].item()
        times = result_datas[trial_i]['times']

        overall_t = result_datas[trial_i]['t'][trajectory_index]
        overall_Y = result_datas[trial_i]['Y'][trajectory_index]

        Ts = result_datas[trial_i]['Ts']
        for collab_type, baseline_obs in obs.items():   
                
                if 'indiv' in collab_type:continue 
                try:
                    mses = get_mses(collab_type, n, overall_t, overall_Y, baseline_obs, Ts, de_type)
                    mse = np.mean(mses)

                    mse_results[collab_type+'-avg-mses'].append(mse)        
                    mse_results[collab_type+'-mses'].append(mses)

                    print(f'**** {collab_type} MSEs: {mses} Avg MSE: {mse} ****')
                except Exception as inst:
                    print(type(inst))    # the exception instance
                    print(inst.args)     # arguments stored in .args
                    print(inst)          # __str__ allows args to be printed directly,
                                         # but may be overridden in exception subclasses

    regression_df = get_regression_df(mse_results)

**** trial index 1 **** 

Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.21432883 0.21690132]
signal variance:       1.3461104682735887
lengthscales:          [0.9438477  1.08880317]
diff signal variance:  0.8305085514857685
diff lengthscales:     [0.89734469]
RMSE over the dimensions is: 3.9371207460941218
name:                  npsde
noise variance:        [0.21432883 0.21690132]
signal variance:       1.3461104682735887
lengthscales:          [0.9438477  1.08880317]
diff signal variance:  0.8305085514857685
diff lengthscales:     [0.89734469]
RMSE over the dimensions is: 3.727436095003661
name:                  npsde
noise variance:        [0.21432883 0.21690132]
signal variance:       1.3461104682735887
lengthscales:          [0.9438477  1.08880317]
diff signal variance:  0.8305085514857685
diff lengthscales:     [0.89734469]
RMSE over the dimensi

RMSE over the dimensions is: 4.013052048219262
name:                  npsde
noise variance:        [0.21085772 0.20742548]
signal variance:       1.1443748894311505
lengthscales:          [1.01281477 1.08083825]
diff signal variance:  0.8403134669510173
diff lengthscales:     [0.87126878]
RMSE over the dimensions is: 3.7726927314296503
name:                  npsde
noise variance:        [0.21085772 0.20742548]
signal variance:       1.1443748894311505
lengthscales:          [1.01281477 1.08083825]
diff signal variance:  0.8403134669510173
diff lengthscales:     [0.87126878]
RMSE over the dimensions is: 6.98800321826565
**** rand_obs MSEs: [4.013052048219262, 3.7726927314296503, 6.98800321826565] Avg MSE: 4.924582665971521 ****
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.19923394 0.19674557]
signal variance:       1.2004068980487435
lengthscales:   

RMSE over the dimensions is: 5.5149382389438895
**** dynamic_beta_obs MSEs: [0.6110410492625463, 3.047201460356969, 5.5149382389438895] Avg MSE: 3.0577269161878013 ****
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.44529949 0.22596995]
signal variance:       1.0481407698694447
lengthscales:          [1.01529337 0.95894985]
diff signal variance:  0.9040212215862713
diff lengthscales:     [0.9649967]
RMSE over the dimensions is: 1.110862418175462
name:                  npsde
noise variance:        [0.44529949 0.22596995]
signal variance:       1.0481407698694447
lengthscales:          [1.01529337 0.95894985]
diff signal variance:  0.9040212215862713
diff lengthscales:     [0.9649967]
RMSE over the dimensions is: 11.52559638870333
name:                  npsde
noise variance:        [0.44529949 0.22596995]
signal variance:       1.0481407698694447
length

RMSE over the dimensions is: 1.8862600200566044
name:                  npsde
noise variance:        [0.28958001 0.31864997]
signal variance:       1.0493953926052533
lengthscales:          [0.97301814 0.954996  ]
diff signal variance:  0.895229121453946
diff lengthscales:     [0.82685987]
RMSE over the dimensions is: 3.9162657728227326
name:                  npsde
noise variance:        [0.28958001 0.31864997]
signal variance:       1.0493953926052533
lengthscales:          [0.97301814 0.954996  ]
diff signal variance:  0.895229121453946
diff lengthscales:     [0.82685987]
RMSE over the dimensions is: 5.778977474660696
**** greedy_sum_obs MSEs: [1.8862600200566044, 3.9162657728227326, 5.778977474660696] Avg MSE: 3.8605010891800107 ****
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.36564136 0.33380581]
signal variance:       1.0016019082474852
lengths

RMSE over the dimensions is: 5.574434913676644
**** greedy_obs_3 MSEs: [2.1752160222346886, 3.853646194984521, 5.574434913676644] Avg MSE: 3.867765710298618 ****
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.37924809 0.28931748]
signal variance:       1.0527942617252333
lengthscales:          [0.97798583 1.07208792]
diff signal variance:  0.901964461684449
diff lengthscales:     [0.92870097]
RMSE over the dimensions is: 3.9039624129638466
name:                  npsde
noise variance:        [0.37924809 0.28931748]
signal variance:       1.0527942617252333
lengthscales:          [0.97798583 1.07208792]
diff signal variance:  0.901964461684449
diff lengthscales:     [0.92870097]
RMSE over the dimensions is: 2.3784628341723892
name:                  npsde
noise variance:        [0.37924809 0.28931748]
signal variance:       1.0527942617252333
lengthscale

RMSE over the dimensions is: 4.516787138281382
name:                  npsde
noise variance:        [0.22666727 0.22609526]
signal variance:       1.0534070303713188
lengthscales:          [0.98333247 1.18563279]
diff signal variance:  0.9081762087603383
diff lengthscales:     [0.93018924]
RMSE over the dimensions is: 2.563893611961305
name:                  npsde
noise variance:        [0.22666727 0.22609526]
signal variance:       1.0534070303713188
lengthscales:          [0.98333247 1.18563279]
diff signal variance:  0.9081762087603383
diff lengthscales:     [0.93018924]
RMSE over the dimensions is: 5.523109053964598
**** greedy_obs_2 MSEs: [4.516787138281382, 2.563893611961305, 5.523109053964598] Avg MSE: 4.201263268069095 ****
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npsde
noise variance:        [0.20700499 0.20856676]
signal variance:       1.1252748549023353
lengthscales

RMSE over the dimensions is: 5.077343687171046
**** entropy_obs MSEs: [4.951289621187042, 3.5623577652919303, 5.077343687171046] Avg MSE: 4.530330357883339 ****


In [20]:
regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,greedy_1,3.961355,0.415532,1.509374,0.133572
1,greedy_2,5.113194,0.781788,2.020337,0.742366
2,greedy_3,4.138763,0.298056,1.320298,0.130503
3,greedy_4,4.198453,0.212938,1.214258,0.234954
4,greedy_sum,4.096240,0.397469,1.898504,0.499957
5,dynamic_beta,3.792370,0.289529,1.553035,0.140785
6,joint,5.679079,0.771345,3.095353,1.374249
7,rand,3.836869,0.411879,1.220882,0.177586
8,entropy,3.361926,0.455779,0.692293,0.205279


In [3]:
n_trials = 5
n = 3
de_type = 'ODE'
trajectory_index = 2

results_dir = oj('results', 'DE')

with cwd(results_dir):
    result_datas = []
    for file in os.listdir():
        if file.endswith('.npz') and de_type.upper() in file:
            result_datas.append(np.load(file, allow_pickle=True))

    mse_results = defaultdict(list)

    for trial_i in range(n_trials):
        print(f'**** trial index {trial_i+1} **** ')
        print()
        obs = result_datas[trial_i]['obs'].item()
        times = result_datas[trial_i]['times']

        overall_t = result_datas[trial_i]['t'][trajectory_index]
        overall_Y = result_datas[trial_i]['Y'][trajectory_index]

        Ts = result_datas[trial_i]['Ts']
        for collab_type, baseline_obs in obs.items():   
                
            if 'indiv' in collab_type: 
                try:
                    mses = get_mses(collab_type, n, overall_t, overall_Y, baseline_obs, Ts, de_type)
                    mse = np.mean(mses)

                    mse_results[collab_type+'-avg-mses'].append(mse)        
                    mse_results[collab_type+'-mses'].append(mses)

                    print(f'**** {collab_type} MSEs: {mses} Avg MSE: {mse} ****')
                except Exception as inst:
                    print(type(inst))    # the exception instance
                    print(inst.args)     # arguments stored in .args
                    print(inst)          # __str__ allows args to be printed directly,
                                         # but may be overridden in exception subclasses

    ode_ind_regression_df = get_regression_df(mse_results)

**** trial index 1 **** 

Model being initialized...



Building loss function...

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.contrib.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use 

RMSE over the dimensions is: 8.265091394012495
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npode
noise variance:        [0.00081491 0.00385079]
signal variance:       1.9007579267037944
lengthscales:          [0.98594686 0.71797308]
RMSE over the dimensions is: 10.750136520276772
**** indiv_greedy_obs MSEs: [3.2090181592693146, 8.265091394012495, 10.750136520276772] Avg MSE: 7.408082024519527 ****
**** trial index 3 **** 

Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization ends.
name:                  npode
noise variance:        [0.00556034 0.0033205 ]
signal variance:       2.8660508982570696
lengthscales:          [0.81520525 1.23615604]
RMSE over the dimensions is: 3.7939577818486914
Model being initialized...
Building loss function...
Adam optimizer being initialized...
Optimization starts.
Optimization 

In [4]:
ode_ind_regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,indiv_greedy,7.270185,0.19833,2.868899,0.080243


In [6]:
with cwd(oj(results_dir, 'regression_results')):
    ode_ind_regression_df.to_csv('ODE-regression-correction.csv', index=False)
    ode_ind_regression_df.to_latex('ODE-regression-correction.tex', index=False)

In [13]:
regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,entropy,2.981224,0.465341,1.157018,0.036469


In [9]:
regression_df # training iter 1000, for entropy and indi

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,entropy,2.864518,0.430831,1.104927,0.051972
1,indiv_greedy,5.179316,0.802955,2.613171,0.137215


In [10]:
with cwd(oj(results_dir, 'regression_results')):
    regression_df.to_csv('SDE-regression-correction.csv', index=False)
    regression_df.to_latex('SDE-regression-correction.tex', index=False)


In [15]:
ODE_regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,greedy_1,5.857746,0.397743,2.359273,0.266774
1,greedy_2,6.725795,0.812513,3.805229,1.075850
2,greedy_3,6.179537,0.635290,2.072478,0.665395
3,greedy_4,5.476039,0.330058,1.987787,0.302376
4,greedy_sum,6.212020,0.661704,2.767850,1.249287
5,dynamic_beta,5.758070,0.424243,2.077605,0.373631
6,joint,5.339084,0.474563,2.125629,0.436403
7,rand,6.239905,0.495719,2.178888,0.432880
8,entropy,5.171592,0.156638,1.920965,0.151457
9,indiv_greedy,6.484443,0.374912,1.281067,0.136480


### VDP

In [20]:
indiv_rand_regression_for_VDP = indiv_rand_regression_df

In [23]:
VDP_mse_results = deepcopy(mse_results)

In [9]:
indiv_rand_regression_for_VDP

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,indiv_greedy,6.839020,0.741066,1.858412,0.265737
1,rand,4.973591,1.254244,2.457056,0.895329


In [18]:
with cwd(oj(local_dir, setting)):
    indiv_rand_regression_df.to_csv('VDP_correction.csv', index=False)

In [7]:

# indiv_regression_df = get_regression_df(mse_results)
indiv_regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,indiv_greedy,6.83902,0.741066,1.858412,0.265737


In [15]:
mse_results

defaultdict(list,
            {'greedy_obs_1-avg-mses': [4.178536317003405,
              4.305602157058645,
              2.868588750859472,
              3.851607596680812,
              5.1480001667037785],
             'greedy_obs_1-mses': [[2.2593988882213423,
               7.560133496191136,
               2.7160765665977378],
              [3.655590891935502, 7.222758576876525, 2.038457002363908],
              [2.2572134940223623, 3.8007284909631096, 2.5478242675929437],
              [2.623772508328433, 3.4927824561882304, 5.438267825525773],
              [1.7120316230317485, 10.138245941169346, 3.5937229359102414]],
             'greedy_obs_2-avg-mses': [4.1246115376366195,
              4.643293169825471,
              4.009888615452311,
              4.359279892610691],
             'greedy_obs_2-mses': [[2.2619341957503094,
               7.441289090976445,
               2.6706113261831033],
              [3.106127457532123, 7.439960984941314, 3.3837910670029743],
     

#### all the share the same target - time [0,8]

In [50]:
de_regression_df

,Baselines,Avg MSE,Stderr,Std MSE,Stderr Std
0,greedy_1,2.756342,NaN,2.756342,NaN
1,greedy_2,2.398316,NaN,2.398316,NaN
2,greedy_3,3.438041,NaN,3.438041,NaN
3,greedy_4,3.627456,NaN,3.627456,NaN
4,greedy_sum,2.944234,NaN,2.944234,NaN
5,dynamic_beta,3.803732,NaN,3.803732,NaN
6,joint,2.785771,NaN,2.785771,NaN
7,rand,2.650305,NaN,2.650305,NaN
8,entropy,2.376445,NaN,2.376445,NaN


In [ ]:
# with cwd(oj(results_dir, setting)):
#     de_regression_df.to_latex('regression_results.tex',index=False)
#     de_regression_df.to_csv('regression_results.csv',index=False)
